In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
import gmaps
import json

import sys
sys.path.append("../")
from location import *

# Keep data consistent, load from disk.
LOAD_DATA_FROM_DISK = True

RAW_FOLDER = 'study_data/'
PROCESSED_FOLDER = 'processed_data/'

In [20]:
file = open(RAW_FOLDER + 'stops_thomas.json', "r")
X = [x.replace('\n', '') for x in file]
json_data = []

for x in X:
    if x: # Check if empty string
        try:
            js = json.loads(x)
            json_data.append(js)
        except:
            None

In [23]:
pd.DataFrame(json_data)

,arrival,centroid,departure,place_id
0,1588631758001,"{'latitude': 55.692229550926186, 'longitude': ...",1588658189001,0
1,1588658191001,"{'latitude': 55.692135452479384, 'longitude': ...",1588659919000,0
2,1588660914001,"{'latitude': 55.69209052332212, 'longitude': 1...",1588664723002,0
3,1588665159143,"{'latitude': 55.692075761056614, 'longitude': ...",1588665365002,0
4,1588665515551,"{'latitude': 55.69205886379219, 'longitude': 1...",1588665727143,0
5,1588665839999,"{'latitude': 55.69210032640927, 'longitude': 1...",1588668688093,0
6,1588668925980,"{'latitude': 55.688916215968696, 'longitude': ...",1588669308002,1
7,1588670379559,"{'latitude': 55.683981459031756, 'longitude': ...",1588670889605,2
8,1588670901670,"{'latitude': 55.683972121916504, 'longitude': ...",1588671183599,2
9,1588671195610,"{'latitude': 55.68395750857616, 'longitude': 1...",1588673138027,2


In [170]:
import gmplot

# Create map orientation centre
lat_mean = stops.latitude.mean()
lon_mean = stops.longitude.mean()

# Create a Gmap Plot object
gmap = gmplot.GoogleMapPlotter(lat_mean, lon_mean, 17)

In [171]:
# Draw places and stops
for place_id in places.place.values:
    # Single place example
    place_radius = 15 # Not the same as meters!
    place_lat = places[places.place == place_id].latitude.values
    place_lon = places[places.place == place_id].longitude.values

    # Plot place with a radius
    gmap.scatter(place_lat, place_lon, color='red', size=place_radius, marker=False)

    # Create a convex hull of the stops
    lats = stops[stops.place == place_id].latitude.values
    lons = stops[stops.place == place_id].longitude.values
    
    # Plot stops as markers
    for lat, lon in zip(lats, lons):
        gmap.marker(lat, lon, color='cornflowerblue')

In [172]:
# Draw map object
gmap.draw('MUC_map.html')